In [1]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.preprocessing import Imputer
from sklearn.model_selection import cross_val_score as cvs
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score as f1
from sklearn.metrics import confusion_matrix, precision_score, recall_score 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVC 
import pandas as pd
from pandas import ExcelFile
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
import scipy.io as io
from sklearn.model_selection import GridSearchCV as gs
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier as knc
from sklearn.ensemble import BaggingClassifier

In [2]:
def show_class(output):
    if output == 1:
        print("Mild asthma")
    elif output == 2:
        print("Moderate asthma")
    else:
        print("Severe asthma")

In [3]:
xl = pd.ExcelFile("sarp.xlsx")
ds = xl.parse('sarp')
X_uncleaned = ds.loc[:, "Baseline_preDrug_FEV1pp":"Baseline_preDrug_FEV1_FVC"]
imputer = Imputer(strategy="median")
imputer.fit(X_uncleaned)
X_cleaned = imputer.transform(X_uncleaned)
X = pd.DataFrame(X_cleaned, columns=X_uncleaned.columns).astype(np.float64)
y = ds.loc[:,"ageasthonset"].astype(np.float64)
stds = ds['status_factorized'].astype(np.int64)
X2 = X
X2 = X["Baseline_preDrug_FEV1_FVC"]
X2 = pd.DataFrame(X2)
X2 = X2.join(y)
X2 = X2.join(ds["gender"])
y2 = stds
y2 = y2.values.ravel()
y2pd = pd.DataFrame(y2)
X2pd = X2
X2 = pd.DataFrame.as_matrix(X2)
X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size=0.33)

In [85]:
knn_clf = Pipeline([("scaler", StandardScaler()),
                    ("knc", knc(weights='uniform', n_jobs=-1, n_neighbors=12))])

In [86]:
knn_clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('knc', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=12, p=2,
           weights='uniform'))])

In [87]:
res = knn_clf.predict([[.72, 15, 1]])
show_class(res[0])

Mild asthma


In [88]:
y_pred = knn_clf.predict(X_test)
y_pred.shape

(449,)

In [89]:
print(confusion_matrix(y_test, y_pred))
print(f1(y_test,y_pred,average='micro'))
print(precision_score(y_test, y_pred, average='micro'))
print(recall_score(y_test, y_pred, average='micro'))

[[125  12  27]
 [ 16  21  52]
 [ 62  13 121]]
0.594654788419
0.594654788419
0.594654788419


In [29]:
param_grid = [ {'knc__n_jobs': [1, 3, 5, 7], 'knc__n_neighbors': [3, 6, 9, 12, 15, 18, 21]}] 
grid_search = gs(knn_clf, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'knc__n_jobs': 1, 'knc__n_neighbors': 18}
Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('knc', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=18, p=2,
           weights='distance'))])


In [15]:
knn_clf = Pipeline([("scaler", StandardScaler()),
                    ("knc", knc(weights='distance', n_jobs=1, n_neighbors=12))])
knn_clf.fit(X_train, y_train)
y_pred = knn_clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(f1(y_test,y_pred,average='micro'))
print(precision_score(y_test, y_pred, average='micro'))
print(recall_score(y_test, y_pred, average='micro'))
X_train.shape

[[104  12  38]
 [ 29  16  56]
 [ 57  34 103]]
0.496659242762
0.496659242762
0.496659242762


(909, 3)

In [17]:
bag_clf = BaggingClassifier(knc(), 
                            n_estimators = 200, max_samples = 909, 
                            bootstrap = True, n_jobs =-1) 
bag_clf.fit(X_train, y_train)
res = bag_clf.predict([[.65, 45, 1]])
show_class(res[0])
y_pred = bag_clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(f1(y_test,y_pred,average='micro'))
print(precision_score(y_test, y_pred, average='micro'))
print(recall_score(y_test, y_pred, average='micro'))

Severe asthma
[[107   9  38]
 [ 41  23  37]
 [ 81  26  87]]
0.483296213808
0.483296213808
0.483296213808
